In [4]:
import pickle
# with open("/scratch/kolubex_emotx_run/data/srt_feats/concat/finetuned_t10/tt0037884/scene-050.ss-0167.es-0167.pkl", 'rb') as f:
with open("/scratch/kolubex_emotx_run/data/srt_feats/independent/finetuned_t10/tt0037884/scene-050.ss-0167.es-0167.pkl", 'rb') as f:
    time = pickle.load(f)
    feats = pickle.load(f)

print(time)
print(feats.shape)

[3.2415, 6.432, 9.1225, 12.042, 15.212]
torch.Size([5, 768])


In [2]:
import numpy as np

with open("/ssd_scratch/cvit/kolubex/data/audio_feats/pretrained/tt0212338/scene-001.ss-0001.es-0013/scene-001.ss-0001.es-0013.npy", 'rb') as f:
    a = np.load(f)
    print(a.shape)
    a = np.squeeze(a, axis=0)
    a = a.transpose((1, 0))
    print(a.shape)

(1, 768, 346)
(346, 768)


In [3]:
import numpy as np
a = (np.arange(1, 10))*1/3
a = a - 0.01
print(a)

[0.32333333 0.65666667 0.99       1.32333333 1.65666667 1.99
 2.32333333 2.65666667 2.99      ]


In [21]:
# this code is to check if all folders that are in /ssd_scratch/cvit/kolubex/data/srt_feats/finetuned_10/ are also in /ssd_scratch/cvit/kolubex/data/audio_feats/pretrained/ or not
# if not then print the folder name
# use glob and os
import glob
import os

path = "/ssd_scratch/cvit/kolubex/data/srt_feats/concat/finetuned_t10/"
checking_path = "/ssd_scratch/cvit/kolubex/data/audio_feats/pretrained/"
# folders = glob.glob(path+"*")
folders = ['tt1568346']
missing_folders = {}
for folder in folders:
    folder_name = folder.split("/")[-1]
    sub_folders = glob.glob(path+folder_name+"/*")
    for sub_folder in sub_folders:
        sub_folder_name = sub_folder.split("/")[-1]
        sub_folder_name = sub_folder_name[:-4]
        # now check if folder/sub_folder_name is in path checking_path or not
        if not os.path.exists(checking_path+folder_name+"/"+sub_folder_name):
            # add the sub_folder_name to missing_folders with the key as folder_name
            if folder_name not in missing_folders:
                missing_folders[folder_name] = [sub_folder_name]
            else:
                missing_folders[folder_name].append(sub_folder_name)
            # print(folder_name + "/" + sub_folder_name)
print(len(missing_folders["tt1568346"]))
# print(len(folders))

173


In [22]:
print(len(missing_folders.keys()))
for key in missing_folders.keys():
    print(len(missing_folders[key]))

1
173
